In [1]:
!pip install gensim

    100% |████████████████████████████████| 22.6MB 1.9MB/s 
    100% |████████████████████████████████| 1.4MB 16.4MB/s 
    100% |████████████████████████████████| 133kB 22.2MB/s 
    100% |████████████████████████████████| 4.4MB 8.6MB/s 
    100% |████████████████████████████████| 61kB 20.0MB/s 
    100% |████████████████████████████████| 552kB 20.2MB/s 
  Running setup.py bdist_wheel for smart-open ... - \ done
  Stored in directory: /content/.cache/pip/wheels/73/f1/9b/ccf93d4ba073b6f79b1ed9df68ab5ce048d8136d0efcf90b30
  Running setup.py bdist_wheel for bz2file ... - done
  Stored in directory: /content/.cache/pip/wheels/81/75/d6/e1317bf09bf1af5a30befc2a007869fa6e1f516b8f7c591cb9
Successfully built smart-open bz2file


In [21]:
import codecs
import glob
import logging
import multiprocessing
import os
import pprint
import re

import nltk
import gensim.models.word2vec as w2v
import sklearn.manifold
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import string
import nltk.data

In [22]:
!ls

datalab  glove.6B.100d.txt  glove.6B.300d.txt  glove.6B.zip
drive	 glove.6B.200d.txt  glove.6B.50d.txt   nltk_data


In [25]:
df = pd.read_csv("drive/app/Misc/whatsapp/analysis/clean_text.csv")

In [26]:
df.head()

,date_time,name,text,cursing,multi
0,2014-01-30 23:04:08,Adamya Rajput,Pics bhejo bc exp 1 and 2 ki,True,False
1,2014-01-30 23:05:45,Bhandu,<‎image omitted>,False,True
2,2014-01-30 23:05:45,Bhandu,<‎image omitted>,False,True
3,2014-01-30 23:05:46,Bhandu,<‎image omitted>,False,True
4,2014-01-30 23:05:46,Bhandu,<‎image omitted>,False,True


In [27]:
## cleaning text with date time which was already cleaned. removing all puncuations except full stop.
def cleaning_text(raw):
 #   raw = raw.translate(None, string.punctuation)
    raw = re.sub(r"\d{2}[/]\d{2}[/]\d{4}","", raw)
    raw = re.sub(r"\d{2}:\d{2} (am|pm)","", raw)
    raw = re.sub(r"\d{1}:\d{2} (am|pm)","", raw)
    
    remove = string.punctuation
    remove = remove.replace(".", "") # don't remove hyphens
    pattern = r"[{}]".format(remove)
    
    clean = re.sub(pattern, "", raw) 
#    clean = re.sub(r'[^\w\s]','',raw)
    return clean

In [28]:
df['col_1'] = df[["text"]].apply(lambda x: cleaning_text(*x), axis=1)

In [29]:
df.head()

,date_time,name,text,cursing,multi,col_1
0,2014-01-30 23:04:08,Adamya Rajput,Pics bhejo bc exp 1 and 2 ki,True,False,Pics bhejo bc exp 1 and 2 ki
1,2014-01-30 23:05:45,Bhandu,<‎image omitted>,False,True,‎image omitted
2,2014-01-30 23:05:45,Bhandu,<‎image omitted>,False,True,‎image omitted
3,2014-01-30 23:05:46,Bhandu,<‎image omitted>,False,True,‎image omitted
4,2014-01-30 23:05:46,Bhandu,<‎image omitted>,False,True,‎image omitted


In [30]:
# calculating length of text. If needed. Not used here.
def cal_len(x):
  lenght = len(x.strip().split(" "))
  return lenght

In [31]:
df['col_2'] = df[["col_1"]].apply(lambda x: cal_len(*x), axis=1)

In [32]:
df.head()

,date_time,name,text,cursing,multi,col_1,col_2
0,2014-01-30 23:04:08,Adamya Rajput,Pics bhejo bc exp 1 and 2 ki,True,False,Pics bhejo bc exp 1 and 2 ki,8
1,2014-01-30 23:05:45,Bhandu,<‎image omitted>,False,True,‎image omitted,2
2,2014-01-30 23:05:45,Bhandu,<‎image omitted>,False,True,‎image omitted,2
3,2014-01-30 23:05:46,Bhandu,<‎image omitted>,False,True,‎image omitted,2
4,2014-01-30 23:05:46,Bhandu,<‎image omitted>,False,True,‎image omitted,2


In [33]:
# removing white spaced from start and end of sentence.
def strip(x):
  stripped = x.strip()
  return stripped

In [34]:
df['col_3'] = df[["col_1"]].apply(lambda x: strip(*x), axis=1)
df.head()

,date_time,name,text,cursing,multi,col_1,col_2,col_3
0,2014-01-30 23:04:08,Adamya Rajput,Pics bhejo bc exp 1 and 2 ki,True,False,Pics bhejo bc exp 1 and 2 ki,8,Pics bhejo bc exp 1 and 2 ki
1,2014-01-30 23:05:45,Bhandu,<‎image omitted>,False,True,‎image omitted,2,‎image omitted
2,2014-01-30 23:05:45,Bhandu,<‎image omitted>,False,True,‎image omitted,2,‎image omitted
3,2014-01-30 23:05:46,Bhandu,<‎image omitted>,False,True,‎image omitted,2,‎image omitted
4,2014-01-30 23:05:46,Bhandu,<‎image omitted>,False,True,‎image omitted,2,‎image omitted


In [35]:
# removing texts of images, videos and links
df = df[df["multi"] == False]
df.head()

,date_time,name,text,cursing,multi,col_1,col_2,col_3
0,2014-01-30 23:04:08,Adamya Rajput,Pics bhejo bc exp 1 and 2 ki,True,False,Pics bhejo bc exp 1 and 2 ki,8,Pics bhejo bc exp 1 and 2 ki
13,2014-01-30 23:06:20,Adamya Rajput,Bhandd bc,True,False,Bhandd bc,2,Bhandd bc
14,2014-01-30 23:06:21,Adamya Rajput,Rukk,False,False,Rukk,1,Rukk
15,2014-01-30 23:06:39,Adamya Rajput,File nai kari kya ?? Kaam ki cheez bhej manual...,False,False,File nai kari kya Kaam ki cheez bhej manual m...,12,File nai kari kya Kaam ki cheez bhej manual m...
16,2014-01-30 23:07:00,Bhandu,Isi se copy kari hai,False,False,Isi se copy kari hai,5,Isi se copy kari hai


In [36]:
clean_list = df["col_3"].tolist()

In [37]:
clean_text = ' '.join(clean_list)

In [38]:
clean_text

'Pics bhejo bc exp 1 and 2 ki Bhandd bc Rukk File nai kari kya  Kaam ki cheez bhej manual matt bhej Isi se copy kari hai Rajjo ko bol Tune nai kari file Abe copies hai Rajjo so gya shaayaf Isi se copy kar le Kauns exp hai Serial order mein hi karna hai Han Acha Bhai meri bhi karde Bsdke meri hui ni Teri kardun Ye hi toh dosti hai Mera second nahi hua Sale tu dikha dosti Mera kar second wala Pehla aur doosra kar de Baaki kar lunga main Tu aisa kar pehle baki karle Pehla dusra hote rahega Check toh vo first 2 hi karega mc Abe to likh Tym kharab mat kar Bc no file here So raha hun Chal kal milte hai Chal so ja Ok Gn Tataa Anybody awake Anyone will be there on time In colleege already college Fucker File karni hai bc Kuch nai kiya Bhai ek ghanta hai lab me class me baith k kar lio Ho jaegi Tweety aa gae Bc aaj meri maarli thi Saali ne Sunky 🐒🐒 Koi aaya hai Tu aa gya Tera match hai aaj Kaunsa match Badminton Main toh ek class attend kar chuka hun Tu pooch raha hai ya bata raha hai Tera matc

In [39]:
# dividing text after after every 500 words. for later use
n = 500
final_text = [clean_text[i:i+n] for i in range(0, len(clean_text), n)]

In [40]:
clean_txt = open("drive/app/Misc/whatsapp/sapp2vec/clean_text.txt", "w")

In [41]:
for item in final_text:
  clean_txt.write("%s\n" % item)
clean_txt.close()